In [13]:
import requests
import pandas as pd

my_headers = {'ZSESSIONID' : '_8lugB2l1SDqbW9UuW54QISHXd6SxEEt0opkiSAMkSrg'}
query = {'start':'1', 'pagesize':'40'}
response = requests.get('https://rally1.rallydev.com/slm/webservice/v2.0/testcase?workspace=https://rally1.rallydev.com/slm/webservice/v2.0/workspace/59461540411&query=(project.name = "NSO-OTE")', headers=my_headers,params=query)
json_data = response.json() if response and response.status_code == 200 else None
print(json_data['QueryResult'].get('TotalResultCount'))

FormattedID,Name,Project,LastRun,LastVerdict,Method,Type,Defects,DefectStatus = [],[],[],[],[],[],[],[],[]

for result in json_data['QueryResult']['Results']:
    test_link = result.get('_ref')
    #print(test_link)
    response_final = requests.get(test_link,headers=my_headers)
    json_data_final = response_final.json() if response_final and response_final.status_code == 200 else None
    fid = json_data_final['TestCase']['FormattedID']
    FormattedID.append(fid)
    
    name = json_data_final['TestCase']['Name']
    Name.append(name)
    
    project = json_data_final['TestCase']['Project']['_refObjectName']
    Project.append(project)
    
    lastrun = json_data_final['TestCase']['LastRun']
    LastRun.append(lastrun)
    
    lastverdict = json_data_final['TestCase']['LastVerdict']
    LastVerdict.append(lastverdict)
    
    method = json_data_final['TestCase']['Method']
    Method.append(method)
    
    ttype = json_data_final['TestCase']['Type']
    Type.append(ttype)
    
    defects = json_data_final['TestCase']['Defects']['Count']
    Defects.append(defects)
    
    defectstatus = json_data_final['TestCase']['DefectStatus']
    DefectStatus.append(defectstatus)
   


33


In [15]:
df = pd.DataFrame(zip(FormattedID,Name,Project,LastRun,LastVerdict,Method,Type,Defects,DefectStatus),columns=['FormattedID','Name','Project','LastRun','LastVerdict','Method','Type','Defects','DefectStatus'])
#df.to_csv('C:/Users/mayjha/Desktop/squad/test.csv',encoding='utf-8', header='true')

df = df.drop(columns=['FormattedID', 'Name'])

def region(row):
    if row['Project'] == 'QA-CoE-Initiatives':
        val = 'AMERICAS'
    elif row['Project'] == 'SAO-Self-Service-Software-Quality-Dashboard':
        val = 'APAC'
    elif row['Project'] == 'NSO-OTE':
        val = 'EMEAR'
    return val

def theatre(row):
    if row['Project'] == 'QA-CoE-Initiatives':
        val = 'America'
    elif row['Project'] == 'SAO-Self-Service-Software-Quality-Dashboard':
        val = 'India'
    elif row['Project'] == 'NSO-OTE':
        val = 'Europe South'
    return val

def quarter(data):
    if data['LastRun'] is not None:
        val = str(data['LastRun'][5:7])
        if val in ['08','09','10']:
            quat = 'Q1FY'+ data['LastRun'][2:4]
        elif val in ['01','11','12']:
            quat = 'Q2FY'+ data['LastRun'][2:4]
        elif val in ['02','03','04']:
            quat = 'Q3FY'+ data['LastRun'][2:4]
        elif val in ['05','06','07']:
            quat = 'Q4FY'+ data['LastRun'][2:4]
        return quat

def Pass(row):
    if row['LastVerdict'] is not None:
        if row['LastVerdict'] == 'Pass':
            val = 1
        else:
            val = 0
        return val

def Fail(row):
    if row['LastVerdict'] is not None:
        if row['LastVerdict'] == 'Fail':
            val = 1
        else:
            val = 0
        return val

def Blocked(row):
    if row['LastVerdict'] == 'Pass' or row['LastVerdict'] == 'Fail':
        val = 0
    else:
        val = 1
    return val

def Testtype(row):
    if row['Type'] == 'Acceptance':
        val = 'Functional'
    elif row['Type'] == 'Performance':
        val = 'Non Functional'
    elif row['Type'] == 'Regression':
        val = 'Functional'
    elif row['Type'] == 'Usability':
        val = 'Functional'
    elif row['Type'] == 'User Interface':
        val = 'Functional'
    elif row['Type'] == 'Functional':
        val = 'Functional'
    return val




df['Region'] = df.apply(region, axis=1)
df['Theatre'] = df.apply(theatre, axis=1)
df['ReportDate'] = df.apply(quarter,axis=1)
df['Passed'] = df.apply(Pass,axis=1)
df['Failed'] = df.apply(Fail,axis=1)
df['Blocked'] = df.apply(Blocked,axis=1)
df['Test Type'] = df.apply(Testtype,axis=1)

df.rename({'Type': 'Test Sub-type'}, axis=1, inplace=True)
df = df.drop(columns=['LastVerdict','LastRun','Defects','DefectStatus'])

# it will drop the duplicate coloumn and keep the count in size column
df = df.groupby(df.columns.tolist(),as_index=False).size()

#copying the actual count of respective coloumn from size
df['Passed'] = df.apply(lambda x: x['size'] if x['Passed'] == 1 else 0, axis=1)
df['Failed'] = df.apply(lambda x: x['size'] if x['Failed'] == 1 else 0, axis=1)
df['Blocked'] = df.apply(lambda x: x['size'] if x['Blocked'] == 1 else 0, axis=1)

#delete the extra size column
df = df.drop(columns=['size'])

df['Passed'] = df.groupby(['Project','Method','Test Sub-type','Region','Theatre','ReportDate','Test Type'])['Passed'].transform('sum')
df['Failed'] = df.groupby(['Project','Method','Test Sub-type','Region','Theatre','ReportDate','Test Type'])['Failed'].transform('sum')
df['Blocked'] = df.groupby(['Project','Method','Test Sub-type','Region','Theatre','ReportDate','Test Type'])['Blocked'].transform('sum')

#calculate quality percentage
def quality(row):
    per = (row['Passed']/(row['Passed']+row['Failed']+row['Blocked']))*100
    return '{:.1f}'.format(per)

df['Quality'] = df.apply(quality,axis=1)

df.drop_duplicates(subset=None, keep='first', inplace=True)


df

,Project,Method,Test Sub-type,Region,Theatre,ReportDate,Passed,Failed,Blocked,Test Type,Quality
0,NSO-OTE,Automated,Acceptance,EMEAR,Europe South,Q3FY21,0,0,1,Functional,0.0
1,NSO-OTE,Manual,Acceptance,EMEAR,Europe South,Q3FY21,32,0,0,Functional,100.0
